In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
sns.set()
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm_notebook

In [2]:
train = pd.read_csv('csi_analyze/dataset/train/subs_csi_train.csv',sep=';')

In [3]:
test = pd.read_csv('csi_analyze/dataset/test/subs_csi_test.csv',sep=';')

In [4]:
train.head(5)

,SK_ID,CSI,CONTACT_DATE
0,2,0,4.05
1,3,0,21.05
2,4,0,27.05
3,5,0,8.05
4,6,1,6.05


In [5]:
train.CSI.value_counts()

0    3566
1     656
Name: CSI, dtype: int64

|name|mean|
|---|---|
|subs_features|В таблице содержится информация о динамике изменения коммерческих характеристик абонентов, прошедших опрос CSI за последний год до опроса. Временная гранулярность данных =  месяц. Уровень агрегации = абонент.|
|subs_bs_consumption|В таблице содержится информация об объеме потребления сервисов (голос, данные на всех сотах) абонентами, прошедшими опрос CSI за последний год  до опроса. Временная гранулярность = месяц Уровень агрегации = абонент / сота|
|subs_bs_data_session|В таблице содержится информация времени и продолжительности DATA-сессий абонентов в привязке к сотам, за последние два месяца до опроса CSI. Временная гранулярность  = сессия. Уровень аграгации =  абонент/сота|
|subs_bs_voice_session|В таблице содержится информация времени и продолжительности VOICE-сессий абонентов в привязке к сотам, за последние два месяца до опроса CSI. Временная гранулярность  = сессия. Уровень аграгации =  абонент/сота|

Features for train

In [7]:
subs_bs_consumption_train = pd.read_csv('csi_analyze/dataset/train/subs_bs_consumption_train.csv',sep=';', decimal=',')

subs_bs_data_session_train = pd.read_csv('csi_analyze/dataset/train/subs_bs_data_session_train.csv',sep=';', decimal=',')
subs_features_train = pd.read_csv('csi_analyze/dataset/train/subs_features_train.csv',sep=';', decimal=',')
subs_bs_voice_session_train = pd.read_csv('csi_analyze/dataset/train/subs_bs_voice_session_train.csv',sep=';', decimal=',')

Features for test

In [8]:
subs_bs_consumption_test = pd.read_csv('csi_analyze/dataset/test/subs_bs_consumption_test.csv',sep=';', decimal=',')

subs_bs_data_session_test = pd.read_csv('csi_analyze/dataset/test/subs_bs_data_session_test.csv',sep=';', decimal=',')
subs_features_test = pd.read_csv('csi_analyze/dataset/test/subs_features_test.csv',sep=';', decimal=',')
subs_bs_voice_session_test = pd.read_csv('csi_analyze/dataset/test/subs_bs_voice_session_test.csv',sep=';', decimal=',')

### subs_bs_consumption - sbc

In [15]:
subs_bs_consumption_train[subs_bs_consumption_train.CELL_LAC_ID == 240277]

,SK_ID,CELL_LAC_ID,MON,SUM_MINUTES,SUM_DATA_MB,SUM_DATA_MIN
0,1827,240277,01.03,0.000000,2.430000e-06,0.000234
773,3852,240277,01.04,0.000000,1.700000e-07,0.000233
855,3852,240277,01.03,0.000069,1.470000e-06,0.000466
141915,3852,240277,01.05,0.000433,1.170000e-06,0.000467
219128,2601,240277,01.05,0.000020,0.000000e+00,0.000000
580851,2601,240277,01.03,0.000264,0.000000e+00,0.000000
943927,4602,240277,01.03,0.000060,0.000000e+00,0.000000


    TODO: взять медиану, создать список из времени и CELL_LAC_ID и заюзать 2 датасета допов

In [32]:
title_mapping = {1.03 : 3, 1.02 : 2, 1.04 : 4, 1.05 : 5}

subs_bs_consumption_train.MON = subs_bs_consumption_train.MON.map(title_mapping)
subs_bs_consumption_test.MON = subs_bs_consumption_test.MON.map(title_mapping)

In [34]:
%%time
sbc_train = []

for user in tqdm_notebook(subs_bs_consumption_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for col in subs_bs_consumption_train.columns[1:]:
        
        if col == 'CELL_LAC_ID':
            #id соты и время 
            cell_list = subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col].values
            #time_list = subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user]['START_TIME'].values
            d['sbc_cell_for_time'] = cell_list
            #d['sbc_time_for_cell'] = time_list
        
        elif col == 'MON':
            #месяц расчёта показателей
            d['mon_mean'] =    subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col].mean()
            d['mon_sum'] =     subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col].sum()
            d['mon_std'] =     subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col].std()
            d['mon_median'] =     subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col].median()
            d['mon_sum_log'] = np.log(subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col].sum())
        
        elif col == 'SUM_MINUTES':
            #суммарный голосовой трафик
            sum_min = subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col]
            sum_min = sum_min.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['sum_min_sum'] =  sum_min.sum()
            d['sum_min_mean'] = sum_min.mean()
            d['sum_min_max'] =  sum_min.max()
            d['sum_min_median'] =  sum_min.median()
            d['sum_min_log_sum'] = np.log(sum_min.sum())
            d['sum_min_std'] =  sum_min.std()
            
            
        elif col == 'SUM_DATA_MB':
            #суммарный трафик данных
            sum_data_mb = subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col]
            sum_data_mb = sum_data_mb.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['sum_data_mb_sum'] = sum_data_mb.sum()
            d['sum_data_mb_mean'] = sum_data_mb.mean()
            d['sum_data_mb_max'] = sum_data_mb.max()
            d['sum_data_mb_median'] = sum_data_mb.median()
            d['sum_data_mb_log_sum'] = np.log(sum_data_mb.sum())
            d['sum_data_mb_std'] = sum_data_mb.std()
            
        elif col == 'SUM_DATA_MIN':
            #суммарная продолжительность потребления данных 
            sum_data_min = subs_bs_consumption_train[subs_bs_consumption_train.SK_ID == user][col]
            sum_data_min = sum_data_min.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['sum_data_min_sum'] = sum_data_min.sum()
            d['sum_data_min_mean'] = sum_data_min.mean()
            d['sum_data_min_max'] = sum_data_min.max()
            d['sum_data_min_median'] = sum_data_min.median()
            d['sum_data_min_log_sum'] = np.log(sum_data_min.sum())
            d['sum_data_min_std'] = sum_data_min.std()
    sbc_train.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 3min 37s, sys: 950 ms, total: 3min 38s
Wall time: 3min 36s


In [35]:
%%time
sbc_test = []

for user in tqdm_notebook(subs_bs_consumption_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for col in subs_bs_consumption_test.columns[1:]:
        
        
        if col == 'CELL_LAC_ID':
            cell_list = subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col].values
            #time_list = subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user]['START_TIME'].values
            d['sbc_cell_for_time'] = cell_list
            #d['sbc_time_for_cell'] = time_list
            
        elif col == 'MON':
            d['mon_mean'] =    subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col].mean()
            d['mon_sum'] =     subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col].sum()
            d['mon_std'] =     subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col].std()
            d['mon_median'] =     subs_bs_consumption_train[subs_bs_consumption_test.SK_ID == user][col].median()
            d['mon_sum_log'] = np.log(subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col].sum())
        
        elif col == 'SUM_MINUTES':
            sum_min = subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col]
            sum_min = sum_min.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['sum_min_sum'] =  sum_min.sum()
            d['sum_min_mean'] = sum_min.mean()
            d['sum_min_max'] =  sum_min.max()
            d['sum_min_median'] =  sum_min.median()
            d['sum_min_log_sum'] = np.log(sum_min.sum())
            d['sum_min_std'] =  sum_min.std()
            
            
        elif col == 'SUM_DATA_MB':
            sum_data_mb = subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col]
            sum_data_mb = sum_data_mb.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['sum_data_mb_sum'] = sum_data_mb.sum()
            d['sum_data_mb_mean'] = sum_data_mb.mean()
            d['sum_data_mb_max'] = sum_data_mb.max()
            d['sum_data_mb_median'] = sum_data_mb.median()
            d['sum_data_mb_log_sum'] = np.log(sum_data_mb.sum())
            d['sum_data_mb_std'] = sum_data_mb.std()
            
        elif col == 'SUM_DATA_MIN':
            sum_data_min = subs_bs_consumption_test[subs_bs_consumption_test.SK_ID == user][col]
            sum_data_min = sum_data_min.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['sum_data_min_sum'] = sum_data_min.sum()
            d['sum_data_min_mean'] = sum_data_min.mean()
            d['sum_data_min_max'] = sum_data_min.max()
            d['sum_data_min_median'] = sum_data_min.median()
            d['sum_data_min_log_sum'] = np.log(sum_data_min.sum())
            d['sum_data_min_std'] = sum_data_min.std()
    sbc_test.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 10min 39s, sys: 2.77 s, total: 10min 41s
Wall time: 10min 38s


In [76]:
print(len(sbc_test), len(sbc_train))

5211 4215


### subs_features - sf

In [218]:
subs_features_train[subs_features_train.SK_ID == 75]['COM_CAT#3']

3189     5
3737     5
8125     5
11269    5
12298    5
16484    5
20734    5
21116    5
25273    5
31692    5
32233    5
37849    5
Name: COM_CAT#3, dtype: int64

In [ ]:
COM_CAT#3

In [206]:
%%time
sf_train = []

for user in tqdm_notebook(subs_features_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for col in subs_features_train.columns[1:]:
        
        #SNAP DATE
        
        if col == "COM_CAT#1":
            com_cat1 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat1 = com_cat1.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat1.unique()) == 1:
                d['com_cat1'] = com_cat1.unique()[0]
            else:
                d['com_cat1_mean'] = com_cat1.mean()
            

        elif col == "COM_CAT#2":
            com_cat2 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat2 = com_cat2.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat2_mean'] = com_cat2.mean()
            d['com_cat2_sum'] = com_cat2.sum()
            d['com_cat2_std'] = com_cat2.std()
            d['com_cat2_median'] = com_cat2.median()
            d['com_cat2_sum_log'] = np.log(com_cat2.sum())

        elif col == "COM_CAT#3":
            com_cat3 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat3 = com_cat3.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            
            if len(com_cat3.unique()) == 1:
                d['com_cat3'] = com_cat3.unique()[0]
            else:
                d['com_cat3_mean'] = com_cat3.mean()
                
#             d['com_cat3_mean'] = com_cat3.mean()
#             d['com_cat3_sum'] = com_cat3.sum()
#             d['com_cat3_std'] = com_cat3.std()
#             d['com_cat3_median'] = com_cat3.median()
#             d['com_cat3_sum_log'] = np.log(com_cat3.sum())

        elif col == "BASE_TYPE":
            #новая или старая абоненская база
            base_type = subs_features_train[subs_features_train.SK_ID == user][col]
            #base_type = base_type.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(base_type.unique()) == 1:
                d['base_type'] = base_type.unique()[0]
            else:
                d['base_type_mean'] = base_type.mean()
            
            
        elif col == "ACT":
            #активность абонента
            act = subs_features_train[subs_features_train.SK_ID == user][col]
            #act = act.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(act.unique()) == 1:
                d['act'] = act.unique()[0]
            else:
                d['act_mean'] = act.mean()
                

        elif col == "ARPU_GROUP":
            #группа по ARPU абонента
            act_group = subs_features_train[subs_features_train.SK_ID == user][col]
            #act_group = act_group.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['act_group_mean'] = act_group.mean()
            d['act_group_sum'] = act_group.sum()
            d['act_group_std'] = act_group.std()
            d['act_group_median'] = act_group.median()
            d['act_group_sum_log'] = np.log(act_group.sum())

        elif col == "COM_CAT#7":
            com_cat7 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat7 = com_cat7.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat7_mean'] = com_cat7.mean()
            d['com_cat7_sum'] = com_cat7.sum()
            d['com_cat7_std'] = com_cat7.std()
            d['com_cat7_median'] = com_cat7.median()
            d['com_cat7_sum_log'] = np.log(com_cat7.sum())

        elif col == "COM_CAT#8":
            com_cat8 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat8 = com_cat8.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat8_mean'] = com_cat8.mean()
            d['com_cat8_sum'] = com_cat8.sum()
            d['com_cat8_std'] = com_cat8.std()
            d['com_cat8_median'] = com_cat8.median()
            d['com_cat8_sum_log'] = np.log(com_cat8.sum())

        elif col == "DEVICE_TYPE_ID":
            #тип устройства
            device_type_id = subs_features_train[subs_features_train.SK_ID == user][col]
            #device_type_id = device_type_id.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(device_type_id.unique()) == 1:
                d['device_type_id'] = device_type_id.unique()[0]
            else:
                d['device_type_id_mean'] = device_type_id.mean()
                
#             d['device_type_id_mean'] = device_type_id.mean()
#             d['device_type_id_sum'] = device_type_id.sum()
#             d['device_type_id_std'] = device_type_id.std()
#             d['device_type_id_median'] = device_type_id.median()
#             d['device_type_id_sum_loрg'] = np.log(device_type_id.sum())

        elif col == "INTERNET_TYPE_ID":
            #технология устр (2G/3G)
            INTERNET_TYPE_ID = subs_features_train[subs_features_train.SK_ID == user][col]
            #INTERNET_TYPE_ID = INTERNET_TYPE_ID.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            
            if len(INTERNET_TYPE_ID.unique()) == 1:
                d['INTERNET_TYPE_ID'] = INTERNET_TYPE_ID.unique()[0]
            else:
                d['INTERNET_TYPE_ID_mean'] = INTERNET_TYPE_ID.mean()
                
#             d['INTERNET_TYPE_ID_mean'] = INTERNET_TYPE_ID.mean()
#             d['INTERNET_TYPE_ID_sum'] = INTERNET_TYPE_ID.sum()
#             d['INTERNET_TYPE_ID_std'] = INTERNET_TYPE_ID.std()
#             d['INTERNET_TYPE_ID_median'] = INTERNET_TYPE_ID.median()
#             d['INTERNET_TYPE_ID_sum_loрg'] = np.log(INTERNET_TYPE_ID.sum())
 
        elif col == "REVENUE":
            #выручка
            REVENUE = subs_features_train[subs_features_train.SK_ID == user][col]
            REVENUE = REVENUE.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['REVENUE_mean'] = REVENUE.mean()
            d['REVENUE_sum'] = REVENUE.sum()
            d['REVENUE_std'] = REVENUE.std()
            d['REVENUE_median'] = REVENUE.median()
            d['REVENUE_sum_loрg'] = np.log(REVENUE.sum())
 
        elif col == "ITC":
            #расходы на интерконнект
            ITC = subs_features_train[subs_features_train.SK_ID == user][col]
            ITC = ITC.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['ITC_mean'] = ITC.mean()
            d['ITC_sum'] = ITC.sum()
            d['ITC_std'] = ITC.std()
            d['ITC_median'] = ITC.median()
            d['ITC_sum_loрg'] = np.log(ITC.sum())
     
        elif col == "VAS":
            #расходы на VAS
            VAS = subs_features_train[subs_features_train.SK_ID == user][col]
            VAS = VAS.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['VAS_mean'] = VAS.mean()
            d['VAS_sum'] = VAS.sum()
            d['VAS_std'] = VAS.std()
            d['VAS_median'] = VAS.median()
            d['VAS_sum_loрg'] = np.log(VAS.sum())
     
        elif col == "RENT_CHANNEL":
            #расходы на аренду каналов
            RENT_CHANNEL = subs_features_train[subs_features_train.SK_ID == user][col]
            RENT_CHANNEL = RENT_CHANNEL.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['RENT_CHANNEL_mean'] = RENT_CHANNEL.mean()
            d['RENT_CHANNEL_sum'] = RENT_CHANNEL.sum()
            d['RENT_CHANNEL_std'] = RENT_CHANNEL.std()
            d['RENT_CHANNEL_median'] = RENT_CHANNEL.median()
            d['RENT_CHANNEL_sum_loрg'] = np.log(RENT_CHANNEL.sum())
            
     
        elif col == "ROAM":
            # раходы на роуминг 
            ROAM = subs_features_train[subs_features_train.SK_ID == user][col]
            ROAM = ROAM.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['ROAM_mean'] = ROAM.mean()
            d['ROAM_sum'] = ROAM.sum()
            d['ROAM_std'] = ROAM.std()
            d['ROAM_median'] = ROAM.median()
            d['ROAM_sum_loрg'] = np.log(ROAM.sum())
     
        elif col == "COST":
            #себестоимость 
            COST = subs_features_train[subs_features_train.SK_ID == user][col]
            COST = COST.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['COST_mean'] = COST.mean()
            d['COST_sum'] = COST.sum()
            d['COST_std'] = COST.std()
            d['COST_median'] = COST.median()
            d['COST_sum_loрg'] = np.log(COST.sum())
     
        elif col == "COM_CAT#17":
            com_cat17 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat17 = com_cat17.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat17_mean'] = com_cat17.mean()
            d['com_cat17_sum'] = com_cat17.sum()
            d['com_cat17_std'] = com_cat17.std()
            d['com_cat17_median'] = com_cat17.median()
            d['com_cat17_sum_log'] = np.log(com_cat17.sum())

        elif col == "COM_CAT#18":
            com_cat18 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat18 = com_cat18.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat18_mean'] = com_cat18.mean()
            d['com_cat18_sum'] = com_cat18.sum()
            d['com_cat18_std'] = com_cat18.std()
            d['com_cat18_median'] = com_cat18.median()
            d['com_cat18_sum_log'] = np.log(com_cat18.sum())

        elif col == "COM_CAT#19":
            com_cat19 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat19 = com_cat19.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat19_mean'] = com_cat19.mean()
            d['com_cat19_sum'] = com_cat19.sum()
            d['com_cat19_std'] = com_cat19.std()
            d['com_cat19_median'] = com_cat19.median()
            d['com_cat19_sum_log'] = np.log(com_cat19.sum())

        elif col == "COM_CAT#20":
            com_cat20 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat20 = com_cat20.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat20_mean'] = com_cat20.mean()
            d['com_cat20_sum'] = com_cat20.sum()
            d['com_cat20_std'] = com_cat20.std()
            d['com_cat20_median'] = com_cat20.median()
            d['com_cat20_sum_log'] = np.log(com_cat20.sum())

        elif col == "COM_CAT#21":
            com_cat21 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat21 = com_cat21.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat21_mean'] = com_cat21.mean()
            d['com_cat21_sum'] = com_cat21.sum()
            d['com_cat21_std'] = com_cat21.std()
            d['com_cat21_median'] = com_cat21.median()
            d['com_cat21_sum_log'] = np.log(com_cat21.sum())

        elif col == "COM_CAT#22":
            com_cat22 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat22 = com_cat22.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat22_mean'] = com_cat22.mean()
            d['com_cat22_sum'] = com_cat22.sum()
            d['com_cat22_std'] = com_cat22.std()
            d['com_cat22_median'] = com_cat22.median()
            d['com_cat22_sum_log'] = np.log(com_cat22.sum())

        elif col == "COM_CAT#23":
            com_cat23 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat23 = com_cat23.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat23_mean'] = com_cat23.mean()
            d['com_cat23_sum'] = com_cat23.sum()
            d['com_cat23_std'] = com_cat23.std()
            d['com_cat23_median'] = com_cat23.median()
            d['com_cat23_sum_log'] = np.log(com_cat23.sum())

        elif col == "COM_CAT#24":
            com_cat24 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat24 = com_cat24.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat24.unique()) == 1:
                d['com_cat24'] = com_cat24.unique()[0]
            else:
                d['com_cat24_mean'] = com_cat24.mean()
#             d['com_cat24_mean'] = com_cat24.mean()
#             d['com_cat24_sum'] = com_cat24.sum()
#             d['com_cat24_std'] = com_cat24.std()
#             d['com_cat24_median'] = com_cat24.median()
#             d['com_cat24_sum_log'] = np.log(com_cat24.sum())
            

        elif col == "COM_CAT#25":
            com_cat25 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat25 = com_cat25.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            
            if len(com_cat25.unique()) == 1:
                d['com_cat25'] = com_cat25.unique()[0]
            else:
                d['com_cat25_mean'] = com_cat25.mean()
                
#             d['com_cat25_mean'] = com_cat25.mean()
#             d['com_cat25_sum'] = com_cat25.sum()
#             d['com_cat25_std'] = com_cat25.std()
#             d['com_cat25_median'] = com_cat25.median()
#             d['com_cat25_sum_log'] = np.log(com_cat25.sum())

        elif col == "COM_CAT#26":
            com_cat26 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat26 = com_cat26.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat26.unique()) == 1:
                d['com_cat26'] = com_cat26.unique()[0]
            else:
                d['com_cat26_mean'] = com_cat26.mean()
                
#             d['com_cat26_mean'] = com_cat26.mean()
#             d['com_cat26_sum'] = com_cat26.sum()
#             d['com_cat26_std'] = com_cat26.std()
#             d['com_cat26_median'] = com_cat26.median()
#             d['com_cat26_sum_log'] = np.log(com_cat26.sum())

        elif col == "COM_CAT#27":
            com_cat27 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat27 = com_cat27.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat27_mean'] = com_cat27.mean()
            d['com_cat27_sum'] = com_cat27.sum()
            d['com_cat27_std'] = com_cat27.std()
            d['com_cat27_median'] = com_cat27.median()
            d['com_cat27_sum_log'] = np.log(com_cat27.sum())

        elif col == "COM_CAT#28":
            com_cat28 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat28 = com_cat28.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat28_mean'] = com_cat28.mean()
            d['com_cat28_sum'] = com_cat28.sum()
            d['com_cat28_std'] = com_cat28.std()
            d['com_cat28_median'] = com_cat28.median()
            d['com_cat28_sum_log'] = np.log(com_cat28.sum())

        elif col == "COM_CAT#29":
            com_cat29 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat29 = com_cat29.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat29_mean'] = com_cat29.mean()
            d['com_cat29_sum'] = com_cat29.sum()
            d['com_cat29_std'] = com_cat29.std()
            d['com_cat29_median'] = com_cat29.median()
            d['com_cat29_sum_log'] = np.log(com_cat29.sum())

        elif col == "COM_CAT#30":
            com_cat30 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat30 = com_cat30.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat30_mean'] = com_cat30.mean()
            d['com_cat30_sum'] = com_cat30.sum()
            d['com_cat30_std'] = com_cat30.std()
            d['com_cat30_median'] = com_cat30.median()
            d['com_cat30_sum_log'] = np.log(com_cat30.sum())

        elif col == "COM_CAT#31":
            com_cat31 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat31 = com_cat31.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat31_mean'] = com_cat31.mean()
            d['com_cat31_sum'] = com_cat31.sum()
            d['com_cat31_std'] = com_cat31.std()
            d['com_cat31_median'] = com_cat31.median()
            d['com_cat31_sum_log'] = np.log(com_cat31.sum())

        elif col == "COM_CAT#32":
            com_cat32 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat32 = com_cat32.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat32_mean'] = com_cat32.mean()
            d['com_cat32_sum'] = com_cat32.sum()
            d['com_cat32_std'] = com_cat32.std()
            d['com_cat32_median'] = com_cat32.median()
            d['com_cat32_sum_log'] = np.log(com_cat32.sum())

        elif col == "COM_CAT#33":
            com_cat33 = subs_features_train[subs_features_train.SK_ID == user][col]
            com_cat33 = com_cat33.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat33_mean'] = com_cat33.mean()
            d['com_cat33_sum'] = com_cat33.sum()
            d['com_cat33_std'] = com_cat33.std()
            d['com_cat33_median'] = com_cat33.median()
            d['com_cat33_sum_log'] = np.log(com_cat33.sum())
            
        elif col == "COM_CAT#34":
            com_cat34 = subs_features_train[subs_features_train.SK_ID == user][col]
            #com_cat34= com_cat34.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat34.unique()) == 1:
                d['com_cat34'] = com_cat34.unique()[0]
            else:
                d['com_cat34_mean'] = com_cat34.mean()
                
#             d['com_cat34_mean'] = com_cat34.mean()
#             d['com_cat34_sum'] = com_cat34.sum()
#             d['com_cat34_std'] = com_cat34.std()
#             d['com_cat34_median'] = com_cat34.median()
#             d['com_cat34_sum_log'] = np.log(com_cat34.sum())        
    sf_train.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 4min 41s, sys: 1.64 s, total: 4min 43s
Wall time: 4min 42s


In [207]:
%%time
sf_test = []

for user in tqdm_notebook(subs_features_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    
    for col in subs_features_test.columns[1:]:
        
        #SNAP DATE
        
        if col == "COM_CAT#1":
            com_cat1 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat1 = com_cat1.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat1.unique()) == 1:
                d['com_cat1'] = com_cat1.unique()[0]
            else:
                d['com_cat1_mean'] = com_cat1.mean()
            

        elif col == "COM_CAT#2":
            com_cat2 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat2 = com_cat2.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat2_mean'] = com_cat2.mean()
            d['com_cat2_sum'] = com_cat2.sum()
            d['com_cat2_std'] = com_cat2.std()
            d['com_cat2_median'] = com_cat2.median()
            d['com_cat2_sum_log'] = np.log(com_cat2.sum())

        elif col == "COM_CAT#3":
            com_cat3 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat3 = com_cat3.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            
            if len(com_cat3.unique()) == 1:
                d['com_cat3'] = com_cat3.unique()[0]
            else:
                d['com_cat3_mean'] = com_cat3.mean()
                
#             d['com_cat3_mean'] = com_cat3.mean()
#             d['com_cat3_sum'] = com_cat3.sum()
#             d['com_cat3_std'] = com_cat3.std()
#             d['com_cat3_median'] = com_cat3.median()
#             d['com_cat3_sum_log'] = np.log(com_cat3.sum())

        elif col == "BASE_TYPE":
            #новая или старая абоненская база
            base_type = subs_features_test[subs_features_test.SK_ID == user][col]
            #base_type = base_type.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(base_type.unique()) == 1:
                d['base_type'] = base_type.unique()[0]
            else:
                d['base_type_mean'] = base_type.mean()
            
            
        elif col == "ACT":
            #активность абонента
            act = subs_features_test[subs_features_test.SK_ID == user][col]
            #act = act.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(act.unique()) == 1:
                d['act'] = act.unique()[0]
            else:
                d['act_mean'] = act.mean()
                

        elif col == "ARPU_GROUP":
            #группа по ARPU абонента
            act_group = subs_features_test[subs_features_test.SK_ID == user][col]
            #act_group = act_group.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['act_group_mean'] = act_group.mean()
            d['act_group_sum'] = act_group.sum()
            d['act_group_std'] = act_group.std()
            d['act_group_median'] = act_group.median()
            d['act_group_sum_log'] = np.log(act_group.sum())

        elif col == "COM_CAT#7":
            com_cat7 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat7 = com_cat7.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat7_mean'] = com_cat7.mean()
            d['com_cat7_sum'] = com_cat7.sum()
            d['com_cat7_std'] = com_cat7.std()
            d['com_cat7_median'] = com_cat7.median()
            d['com_cat7_sum_log'] = np.log(com_cat7.sum())

        elif col == "COM_CAT#8":
            com_cat8 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat8 = com_cat8.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat8_mean'] = com_cat8.mean()
            d['com_cat8_sum'] = com_cat8.sum()
            d['com_cat8_std'] = com_cat8.std()
            d['com_cat8_median'] = com_cat8.median()
            d['com_cat8_sum_log'] = np.log(com_cat8.sum())

        elif col == "DEVICE_TYPE_ID":
            #тип устройства
            device_type_id = subs_features_test[subs_features_test.SK_ID == user][col]
            #device_type_id = device_type_id.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(device_type_id.unique()) == 1:
                d['device_type_id'] = device_type_id.unique()[0]
            else:
                d['device_type_id_mean'] = device_type_id.mean()
                
#             d['device_type_id_mean'] = device_type_id.mean()
#             d['device_type_id_sum'] = device_type_id.sum()
#             d['device_type_id_std'] = device_type_id.std()
#             d['device_type_id_median'] = device_type_id.median()
#             d['device_type_id_sum_loрg'] = np.log(device_type_id.sum())

        elif col == "INTERNET_TYPE_ID":
            #технология устр (2G/3G)
            INTERNET_TYPE_ID = subs_features_test[subs_features_test.SK_ID == user][col]
            #INTERNET_TYPE_ID = INTERNET_TYPE_ID.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            
            if len(INTERNET_TYPE_ID.unique()) == 1:
                d['INTERNET_TYPE_ID'] = INTERNET_TYPE_ID.unique()[0]
            else:
                d['INTERNET_TYPE_ID_mean'] = INTERNET_TYPE_ID.mean()
                
#             d['INTERNET_TYPE_ID_mean'] = INTERNET_TYPE_ID.mean()
#             d['INTERNET_TYPE_ID_sum'] = INTERNET_TYPE_ID.sum()
#             d['INTERNET_TYPE_ID_std'] = INTERNET_TYPE_ID.std()
#             d['INTERNET_TYPE_ID_median'] = INTERNET_TYPE_ID.median()
#             d['INTERNET_TYPE_ID_sum_loрg'] = np.log(INTERNET_TYPE_ID.sum())
 
        elif col == "REVENUE":
            #выручка
            REVENUE = subs_features_test[subs_features_test.SK_ID == user][col]
            REVENUE = REVENUE.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['REVENUE_mean'] = REVENUE.mean()
            d['REVENUE_sum'] = REVENUE.sum()
            d['REVENUE_std'] = REVENUE.std()
            d['REVENUE_median'] = REVENUE.median()
            d['REVENUE_sum_loрg'] = np.log(REVENUE.sum())
 
        elif col == "ITC":
            #расходы на интерконнект
            ITC = subs_features_test[subs_features_test.SK_ID == user][col]
            ITC = ITC.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['ITC_mean'] = ITC.mean()
            d['ITC_sum'] = ITC.sum()
            d['ITC_std'] = ITC.std()
            d['ITC_median'] = ITC.median()
            d['ITC_sum_loрg'] = np.log(ITC.sum())
     
        elif col == "VAS":
            #расходы на VAS
            VAS = subs_features_test[subs_features_test.SK_ID == user][col]
            VAS = VAS.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['VAS_mean'] = VAS.mean()
            d['VAS_sum'] = VAS.sum()
            d['VAS_std'] = VAS.std()
            d['VAS_median'] = VAS.median()
            d['VAS_sum_loрg'] = np.log(VAS.sum())
     
        elif col == "RENT_CHANNEL":
            #расходы на аренду каналов
            RENT_CHANNEL = subs_features_test[subs_features_test.SK_ID == user][col]
            RENT_CHANNEL = RENT_CHANNEL.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['RENT_CHANNEL_mean'] = RENT_CHANNEL.mean()
            d['RENT_CHANNEL_sum'] = RENT_CHANNEL.sum()
            d['RENT_CHANNEL_std'] = RENT_CHANNEL.std()
            d['RENT_CHANNEL_median'] = RENT_CHANNEL.median()
            d['RENT_CHANNEL_sum_loрg'] = np.log(RENT_CHANNEL.sum())
            
     
        elif col == "ROAM":
            # раходы на роуминг 
            ROAM = subs_features_test[subs_features_test.SK_ID == user][col]
            ROAM = ROAM.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['ROAM_mean'] = ROAM.mean()
            d['ROAM_sum'] = ROAM.sum()
            d['ROAM_std'] = ROAM.std()
            d['ROAM_median'] = ROAM.median()
            d['ROAM_sum_loрg'] = np.log(ROAM.sum())
     
        elif col == "COST":
            #себестоимость 
            COST = subs_features_test[subs_features_test.SK_ID == user][col]
            COST = COST.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['COST_mean'] = COST.mean()
            d['COST_sum'] = COST.sum()
            d['COST_std'] = COST.std()
            d['COST_median'] = COST.median()
            d['COST_sum_loрg'] = np.log(COST.sum())
     
        elif col == "COM_CAT#17":
            com_cat17 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat17 = com_cat17.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat17_mean'] = com_cat17.mean()
            d['com_cat17_sum'] = com_cat17.sum()
            d['com_cat17_std'] = com_cat17.std()
            d['com_cat17_median'] = com_cat17.median()
            d['com_cat17_sum_log'] = np.log(com_cat17.sum())

        elif col == "COM_CAT#18":
            com_cat18 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat18 = com_cat18.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat18_mean'] = com_cat18.mean()
            d['com_cat18_sum'] = com_cat18.sum()
            d['com_cat18_std'] = com_cat18.std()
            d['com_cat18_median'] = com_cat18.median()
            d['com_cat18_sum_log'] = np.log(com_cat18.sum())

        elif col == "COM_CAT#19":
            com_cat19 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat19 = com_cat19.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat19_mean'] = com_cat19.mean()
            d['com_cat19_sum'] = com_cat19.sum()
            d['com_cat19_std'] = com_cat19.std()
            d['com_cat19_median'] = com_cat19.median()
            d['com_cat19_sum_log'] = np.log(com_cat19.sum())

        elif col == "COM_CAT#20":
            com_cat20 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat20 = com_cat20.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat20_mean'] = com_cat20.mean()
            d['com_cat20_sum'] = com_cat20.sum()
            d['com_cat20_std'] = com_cat20.std()
            d['com_cat20_median'] = com_cat20.median()
            d['com_cat20_sum_log'] = np.log(com_cat20.sum())

        elif col == "COM_CAT#21":
            com_cat21 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat21 = com_cat21.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat21_mean'] = com_cat21.mean()
            d['com_cat21_sum'] = com_cat21.sum()
            d['com_cat21_std'] = com_cat21.std()
            d['com_cat21_median'] = com_cat21.median()
            d['com_cat21_sum_log'] = np.log(com_cat21.sum())

        elif col == "COM_CAT#22":
            com_cat22 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat22 = com_cat22.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat22_mean'] = com_cat22.mean()
            d['com_cat22_sum'] = com_cat22.sum()
            d['com_cat22_std'] = com_cat22.std()
            d['com_cat22_median'] = com_cat22.median()
            d['com_cat22_sum_log'] = np.log(com_cat22.sum())

        elif col == "COM_CAT#23":
            com_cat23 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat23 = com_cat23.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat23_mean'] = com_cat23.mean()
            d['com_cat23_sum'] = com_cat23.sum()
            d['com_cat23_std'] = com_cat23.std()
            d['com_cat23_median'] = com_cat23.median()
            d['com_cat23_sum_log'] = np.log(com_cat23.sum())

        elif col == "COM_CAT#24":
            com_cat24 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat24 = com_cat24.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat24.unique()) == 1:
                d['com_cat24'] = com_cat24.unique()[0]
            else:
                d['com_cat24_mean'] = com_cat24.mean()
#             d['com_cat24_mean'] = com_cat24.mean()
#             d['com_cat24_sum'] = com_cat24.sum()
#             d['com_cat24_std'] = com_cat24.std()
#             d['com_cat24_median'] = com_cat24.median()
#             d['com_cat24_sum_log'] = np.log(com_cat24.sum())
            

        elif col == "COM_CAT#25":
            com_cat25 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat25 = com_cat25.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            
            if len(com_cat25.unique()) == 1:
                d['com_cat25'] = com_cat25.unique()[0]
            else:
                d['com_cat25_mean'] = com_cat25.mean()
                
#             d['com_cat25_mean'] = com_cat25.mean()
#             d['com_cat25_sum'] = com_cat25.sum()
#             d['com_cat25_std'] = com_cat25.std()
#             d['com_cat25_median'] = com_cat25.median()
#             d['com_cat25_sum_log'] = np.log(com_cat25.sum())

        elif col == "COM_CAT#26":
            com_cat26 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat26 = com_cat26.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat26.unique()) == 1:
                d['com_cat26'] = com_cat26.unique()[0]
            else:
                d['com_cat26_mean'] = com_cat26.mean()
                
#             d['com_cat26_mean'] = com_cat26.mean()
#             d['com_cat26_sum'] = com_cat26.sum()
#             d['com_cat26_std'] = com_cat26.std()
#             d['com_cat26_median'] = com_cat26.median()
#             d['com_cat26_sum_log'] = np.log(com_cat26.sum())

        elif col == "COM_CAT#27":
            com_cat27 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat27 = com_cat27.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat27_mean'] = com_cat27.mean()
            d['com_cat27_sum'] = com_cat27.sum()
            d['com_cat27_std'] = com_cat27.std()
            d['com_cat27_median'] = com_cat27.median()
            d['com_cat27_sum_log'] = np.log(com_cat27.sum())

        elif col == "COM_CAT#28":
            com_cat28 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat28 = com_cat28.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat28_mean'] = com_cat28.mean()
            d['com_cat28_sum'] = com_cat28.sum()
            d['com_cat28_std'] = com_cat28.std()
            d['com_cat28_median'] = com_cat28.median()
            d['com_cat28_sum_log'] = np.log(com_cat28.sum())

        elif col == "COM_CAT#29":
            com_cat29 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat29 = com_cat29.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat29_mean'] = com_cat29.mean()
            d['com_cat29_sum'] = com_cat29.sum()
            d['com_cat29_std'] = com_cat29.std()
            d['com_cat29_median'] = com_cat29.median()
            d['com_cat29_sum_log'] = np.log(com_cat29.sum())

        elif col == "COM_CAT#30":
            com_cat30 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat30 = com_cat30.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat30_mean'] = com_cat30.mean()
            d['com_cat30_sum'] = com_cat30.sum()
            d['com_cat30_std'] = com_cat30.std()
            d['com_cat30_median'] = com_cat30.median()
            d['com_cat30_sum_log'] = np.log(com_cat30.sum())

        elif col == "COM_CAT#31":
            com_cat31 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat31 = com_cat31.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat31_mean'] = com_cat31.mean()
            d['com_cat31_sum'] = com_cat31.sum()
            d['com_cat31_std'] = com_cat31.std()
            d['com_cat31_median'] = com_cat31.median()
            d['com_cat31_sum_log'] = np.log(com_cat31.sum())

        elif col == "COM_CAT#32":
            com_cat32 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat32 = com_cat32.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat32_mean'] = com_cat32.mean()
            d['com_cat32_sum'] = com_cat32.sum()
            d['com_cat32_std'] = com_cat32.std()
            d['com_cat32_median'] = com_cat32.median()
            d['com_cat32_sum_log'] = np.log(com_cat32.sum())

        elif col == "COM_CAT#33":
            com_cat33 = subs_features_test[subs_features_test.SK_ID == user][col]
            com_cat33 = com_cat33.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['com_cat33_mean'] = com_cat33.mean()
            d['com_cat33_sum'] = com_cat33.sum()
            d['com_cat33_std'] = com_cat33.std()
            d['com_cat33_median'] = com_cat33.median()
            d['com_cat33_sum_log'] = np.log(com_cat33.sum())
            
        elif col == "COM_CAT#34":
            com_cat34 = subs_features_test[subs_features_test.SK_ID == user][col]
            #com_cat34= com_cat34.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            if len(com_cat34.unique()) == 1:
                d['com_cat34'] = com_cat34.unique()[0]
            else:
                d['com_cat34_mean'] = com_cat34.mean()
                
#             d['com_cat34_mean'] = com_cat34.mean()
#             d['com_cat34_sum'] = com_cat34.sum()
#             d['com_cat34_std'] = com_cat34.std()
#             d['com_cat34_median'] = com_cat34.median()
#             d['com_cat34_sum_log'] = np.log(com_cat34.sum())        
    sf_test.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 5min 58s, sys: 2.37 s, total: 6min
Wall time: 5min 58s


In [11]:
test.columns = ['user_id', 'CONTACT_DATE']

In [12]:
train.columns = ['user_id', 'CSI', 'CONTACT_DATE']

In [208]:
sf_test = pd.DataFrame(sf_test)
sf_train = pd.DataFrame(sf_train)
sbc_test = pd.DataFrame(sbc_test)
sbc_train = pd.DataFrame(sbc_train)

    TODO: checkout

In [19]:
sf_sbc_train = pd.merge(train, sbc_train, on=['user_id'],how='outer')

In [20]:
sf_sbc_train = pd.merge(sf_sbc_train, sf_train, on=['user_id'],how='outer')

In [21]:
sf_sbc_train.to_csv('data_ready_train/sf_sbc_train.csv',index=False)

In [22]:
sf_sbc_test = pd.merge(test,sbc_test, on=['user_id'],how='outer')

In [23]:
sf_sbc_test = pd.merge(sf_sbc_test, sf_test, on=['user_id'], how='outer')

In [24]:
sf_sbc_test.to_csv('data_ready_train/sf_sbc_test.csv', index=False)

### subs_bs_data_session - sbds

In [107]:
import tqdm

In [58]:
subs_bs_data_session_train['START_TIME'] = subs_bs_data_session_train['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')
subs_bs_data_session_test['START_TIME'] = subs_bs_data_session_test['START_TIME'].map(lambda x: x.split()[0].split('.')[1], na_action='ignore')

In [109]:
%%time
sbds_train = []

for user in subs_bs_data_session_train.SK_ID.unique():
    d = {}
    d['user_id'] = user
    
    for month in subs_bs_data_session_train[subs_bs_data_session_train.SK_ID == user]['START_TIME'].unique():
        se = subs_bs_data_session_train[subs_bs_data_session_train.SK_ID == user][subs_bs_data_session_train.START_TIME.astype('str') == '{}'.format(month)]
        for col in subs_bs_data_session_train.columns[1:]:

            if col == 'CELL_LAC_ID':    
                d['cell_id_' + month] = se['CELL_LAC_ID'].values
        
            elif col == 'DATA_VOL_MB':
                d['data_vol_mb_mean_'+ month] = se['DATA_VOL_MB'].mean()
                d['data_vol_mb_sum_'+ month] = se['DATA_VOL_MB'].sum()
                d['data_vol_mb_std_'+ month] = se['DATA_VOL_MB'].std()
                #d['data_vol_mb_log_sum_'+ month] = np.log(se.sum())
                d['data_vol_mb_median_'+ month] = se['DATA_VOL_MB'].median()
                d['data_vol_mb_min_'+ month] = se['DATA_VOL_MB'].min()
                d['data_vol_mb_max_'+ month] = se['DATA_VOL_MB'].max()
                
    sbds_train.append(d)

KeyboardInterrupt: 

In [110]:
sbds_train

[]

In [239]:
# %%time
# sbds_train = []

# for user in tqdm_notebook(subs_bs_data_session_train.SK_ID.unique()):
#     d = {}
#     d['user_id'] = user
#     for col in subs_bs_data_session_train.columns[1:]:
        
#         if col == 'CELL_LAC_ID':
#             cell_list = subs_bs_data_session_train[subs_bs_data_session_train.SK_ID == user][col].values
#             time_list = subs_bs_data_session_train[subs_bs_data_session_train.SK_ID == user]['START_TIME'].values
#             d['cell_for_time'] = cell_list
#             d['time_for_cell'] = time_list
            
#         elif col == 'DATA_VOL_MB':
#             #кол-во потребления данных за одну сессию
#             data_vol_mb = subs_bs_data_session_train[subs_bs_data_session_train.SK_ID == user][col]
#             data_vol_mb = data_vol_mb.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
#             d['data_vol_mb_mean'] = data_vol_mb.mean()
#             d['data_vol_mb_sum'] =  data_vol_mb.sum()
#             d['data_vol_mb_std'] =  data_vol_mb.std()
#             d['data_vol_mb_sum_log'] = np.log(data_vol_mb.sum())
#             d['data_vol_mb_med'] =  data_vol_mb.median()
#             d['data_vol_mb_min'] =  data_vol_mb.min()
#             d['data_vol_mb_max'] =  data_vol_mb.max()
#     sbds_train.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 2min 1s, sys: 411 ms, total: 2min 1s
Wall time: 2min 1s


In [240]:
%%time
sbds_test = []

for user in tqdm_notebook(subs_bs_data_session_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for col in subs_bs_data_session_test.columns[1:]:
        
        if col == 'CELL_LAC_ID':
            cell_list = subs_bs_data_session_test[subs_bs_data_session_test.SK_ID == user][col].values
            time_list = subs_bs_data_session_test[subs_bs_data_session_test.SK_ID == user]['START_TIME'].values
            d['cell_for_time'] = cell_list
            d['time_for_cell'] = time_list
            
        elif col == 'DATA_VOL_MB':
            data_vol_mb = subs_bs_data_session_test[subs_bs_data_session_test.SK_ID == user][col]
            data_vol_mb = data_vol_mb.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['data_vol_mb_mean'] = data_vol_mb.mean()
            d['data_vol_mb_sum'] =  data_vol_mb.sum()
            d['data_vol_mb_std'] =  data_vol_mb.std()
            d['data_vol_mb_sum_log'] = np.log(data_vol_mb.sum())
            d['data_vol_mb_med'] =  data_vol_mb.median()
            d['data_vol_mb_min'] =  data_vol_mb.min()
            d['data_vol_mb_max'] =  data_vol_mb.max()
    sbds_test.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 2min 47s, sys: 665 ms, total: 2min 47s
Wall time: 2min 47s


In [241]:
sbds_train = pd.DataFrame(sbds_train)
sbds_test = pd.DataFrame(sbds_test)

### subs_bs_voice_session - sbvs

In [242]:
%%time
sbvs_train = []

for user in tqdm_notebook(subs_bs_voice_session_train.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for col in subs_bs_voice_session_train.columns[1:]:
        
        if col == 'CELL_LAC_ID':
            cell_list = subs_bs_voice_session_train[subs_bs_voice_session_train.SK_ID == user][col].values
            time_list = subs_bs_voice_session_train[subs_bs_voice_session_train.SK_ID == user]['START_TIME'].values
            d['v_cell_for_time'] = cell_list
            d['v_time_for_cell'] = time_list
            
        elif col == 'VOICE_DUR_MIN':
            data_vol_mb = subs_bs_voice_session_train[subs_bs_voice_session_train.SK_ID == user][col]
            data_vol_mb = data_vol_mb.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['VOICE_DUR_MIN_mean'] = data_vol_mb.mean()
            d['VOICE_DUR_MIN_sum'] =  data_vol_mb.sum()
            d['VOICE_DUR_MIN_std'] =  data_vol_mb.std()
            d['VOICE_DUR_MIN_sum_log'] = np.log(data_vol_mb.sum())
            d['VOICE_DUR_MIN_med'] =  data_vol_mb.median()
            d['VOICE_DUR_MIN_min'] =  data_vol_mb.min()
            d['VOICE_DUR_MIN_max'] =  data_vol_mb.max()
    sbvs_train.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 1min 34s, sys: 436 ms, total: 1min 34s
Wall time: 1min 34s


In [243]:
%%time
sbvs_test = []

for user in tqdm_notebook(subs_bs_voice_session_test.SK_ID.unique()):
    d = {}
    d['user_id'] = user
    for col in subs_bs_voice_session_test.columns[1:]:
        
        if col == 'CELL_LAC_ID':
            cell_list = subs_bs_voice_session_test[subs_bs_voice_session_test.SK_ID == user][col].values
            time_list = subs_bs_voice_session_test[subs_bs_voice_session_test.SK_ID == user]['START_TIME'].values
            d['v_cell_for_time'] = cell_list
            d['v_time_for_cell'] = time_list
            
        elif col == 'VOICE_DUR_MIN':
            data_vol_mb = subs_bs_voice_session_test[subs_bs_voice_session_test.SK_ID == user][col]
            data_vol_mb = data_vol_mb.map(lambda x: float(x.replace(',','0.')), na_action='ignore')
            d['VOICE_DUR_MIN_mean'] = data_vol_mb.mean()
            d['VOICE_DUR_MIN_sum'] =  data_vol_mb.sum()
            d['VOICE_DUR_MIN_std'] =  data_vol_mb.std()
            d['VOICE_DUR_MIN_sum_log'] = np.log(data_vol_mb.sum())
            d['VOICE_DUR_MIN_med'] =  data_vol_mb.median()
            d['VOICE_DUR_MIN_min'] =  data_vol_mb.min()
            d['VOICE_DUR_MIN_max'] =  data_vol_mb.max()
    sbvs_test.append(d)

Widget Javascript not detected.  It may not be installed or enabled properly.



CPU times: user 2min 2s, sys: 680 ms, total: 2min 2s
Wall time: 2min 2s


In [244]:
sbvs_train = pd.DataFrame(sbvs_train)
sbvs_test = pd.DataFrame(sbvs_test)

In [245]:
sf_test.to_csv('data_ready_train/sf_test.csv', index=False)
sf_train.to_csv('data_ready_train/sf_train.csv', index=False)

sbc_test.to_csv('data_ready_train/sbc_test.csv', index=False)
sbc_train.to_csv('data_ready_train/sbc_train.csv', index=False)

sbds_train.to_csv('data_ready_train/sbds_train.csv', index=False)
sbds_test.to_csv('data_ready_train/sbds_test.csv', index=False)

sbvs_train.to_csv('data_ready_train/sbvs_train.csv', index=False)
sbvs_test.to_csv('data_ready_train/sbvs_test.csv', index=False)

### bs_avg_kpi - bak | bs_chnn_kpi - bck

In [255]:
sbds_train[sbds_train.user_id == 3853].cell_for_time.values[0][0]

61951

In [256]:
sbds_train[sbds_train.user_id == 3853].time_for_cell.values[0][0]

'04.04 05:00:00'

In [6]:
bak = pd.read_csv('csi_analyze/dataset/bs_avg_kpi.csv',sep=';', nrows=10)

In [7]:
columns = bak.columns

In [9]:
col = {}
for i in columns:
    col[i] = np.float
col['T_DATE'] = np.str
col['CELL_LAC_ID'] = np.int32

In [ ]:
%%time
bak = pd.read_csv('csi_analyze/dataset/bs_avg_kpi.csv',sep=';',  decimal=',', dtype=col)

In [ ]:
df